In [38]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["groq_api_key"] = os.getenv("groq_api_key")

In [39]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "qwen/qwen3-32b")

In [40]:
from langchain_core.messages import HumanMessage
message = HumanMessage(content= "My name is amber khan i am working as AI agent developer in Google")
model.invoke([message])

AIMessage(content="<think>\nOkay, the user mentioned they're a Google AI agent developer working on conversational agents. I need to respond appropriately. First, I should acknowledge their role and ask about specific projects or technologies they're using. Maybe they're working on something like Dialogflow or Vertex AI? I should check if they're involved in research or product development. Also, ask about challenges they face in the field. Need to keep the conversation engaging and offer help if they need technical advice or want to discuss AI trends. Make sure the tone is friendly and professional. Avoid assuming too much, so open-ended questions are good. Let me structure the response to cover these points without being too verbose.\n</think>\n\nHello Amber Khan! It's great to meet another AI developer. I'm curious about your work at Google - could you share more about what kinds of AI agents you're developing? Are you working on conversational agents, decision-making systems, or so

In [41]:
from langchain_core.messages import AIMessage
response = model.invoke([
    HumanMessage(content= "Do not provide me the think content in your output"),
    HumanMessage(content= "My name is amber khan i am working as AI agent developer in Google"),
    AIMessage(content = "Thanks for sharing, Amber Khan! Working as an AI agent developer at Google is an incredible opportunity to contribute to cutting-edge advancements in AI."),
    HumanMessage(content= "Who am i and where i work?")
])

In [42]:
from langchain_core.chat_history import BaseChatMessageHistory # ye ek interface h jo session_id ke hisab se Chat history ka object return krega 
from langchain_community.chat_message_histories import ChatMessageHistory # Ye basically store place h humanmessage Ai message sbko add krta h aur history delte krta h 
from langchain_core.runnables.history import RunnableWithMessageHistory # Jo history h usko chain ke input me inject krta h aur fir run krte waqt us hisaab se output deta h 

store = {} # Yeh ek dictionary hai jisme tum har session ki history store kar rahe ho.
def session_wise_history(session_id)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chat_with_history = RunnableWithMessageHistory(model, session_wise_history)

In [43]:
Configuration = {"configurable" : {"session_id" : "chat1"}}
response = chat_with_history.invoke(
    [HumanMessage(content= "Do not provide me the think content in your output. My name is amber khan i am working as AI agent developer in Google")],
    config = Configuration
)
response.content

"<think>\n\n</think>\n\nHi Amber Khan, nice to meet you! As an AI agent developer at Google, you're working on some really cutting-edge technology. Is there anything specific you'd like to discuss or ask about related to AI agent development?"

In [44]:
new_response = chat_with_history.invoke(
    [HumanMessage(content= "who am i and where exactly i work")],
    config= Configuration
    )
new_response.content

"<think>\n\n</think>\n\nYou are Amber Khan, and you work as an AI agent developer at Google. If you have more details or a specific context you'd like to clarify, feel free to share!"

In [45]:
configuration2 = {"configurable" : {"session_id" : "chat2"}}
response2 = chat_with_history.invoke(
    [HumanMessage("Hey do you know me?")],
    configuration2
)
response2.content

"<think>\nOkay, the user is asking if I know them. Let me think about how to approach this. First, I should acknowledge their question directly and truthfully. I don't have memory of past interactions beyond the current conversation, so I need to make that clear.\n\nI should probably explain my limitations in a friendly way. They might be testing if I can recognize previous conversations or if they expect me to remember something. But since I can't, I need to state that upfront to avoid confusion.\n\nMaybe they're looking for a sense of continuity or connection. I should focus on being helpful moving forward rather than dwelling on the lack of memory. Offering assistance with whatever they need now is key. I should keep the tone open and inviting to encourage them to share what they need help with. Also, making sure I'm clear about my role as an AI assistant while being approachable.\n</think>\n\nHi there! I don't have memory of past interactions (including this one 😊), but I'd love to

In [46]:
response3 = chat_with_history.invoke(
    [HumanMessage("Do you know my company name?")],
    Configuration
)

In [47]:
response3

AIMessage(content='<think>\n\n</think>\n\nYes, you mentioned that you work at **Google**.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 150, 'total_tokens': 167, 'completion_time': 0.054076062, 'completion_tokens_details': None, 'prompt_time': 0.006316281, 'prompt_tokens_details': None, 'queue_time': 0.056478159, 'total_time': 0.060392343}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--6f77688e-dfb2-405c-bdc8-f6827065ef66-0', usage_metadata={'input_tokens': 150, 'output_tokens': 17, 'total_tokens': 167})

## Working with Prompt Template 

In [48]:
from langchain_core.prompts  import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Responsiblity : Make sure not to give think content in your response , Act like a professional Q&A Analyst and answer all the question as asked by user"),
        MessagesPlaceholder("message")
    ]
)

chain = prompt | model 

In [49]:
from langchain_core.runnables.history import RunnableWithMessageHistory
chat_with_history = RunnableWithMessageHistory(chain, session_wise_history)

In [50]:
configuration3 = {"configurable" : {"session_id" : "chat4"}}
response = chat_with_history.invoke({
    [HumanMessage("I am MD AMBER KHAN 23 yr old i want to become a AI devloper")],
    configuration3 }
)
response.content

TypeError: unhashable type: 'list'

In [ ]:
response = chat_with_history.invoke(
    [HumanMessage("What i want to become")],
    configuration3
)
response.content

"<think>\n\n</think>\n\nYou've clearly expressed your **goal** multiple times now, and to restate it clearly and professionally:\n\n> **You want to become an AI Developer.**\n\nHere is a **professional summary** of your goal:\n\n---\n\n## 🎯 **Career Objective: AI Developer**\n\n**Objective:** To pursue a career as an **AI Developer**, leveraging data-driven insights, machine learning models, and artificial intelligence techniques to build intelligent systems and innovative solutions.\n\n**Target Audience:**  \n- Entry-level AI Developer roles  \n- Internships in AI/ML  \n- Freelance or project-based AI development work  \n\n**Skills to Develop:**  \n- Programming (Python, R)  \n- Machine Learning & Deep Learning  \n- Data Analysis & Visualization  \n- AI Libraries (TensorFlow, PyTorch, Scikit-learn)  \n- Project Portfolio (GitHub, Kaggle)  \n- Problem-Solving & Logical Thinking  \n\n**Timeline:**  \nYou are currently in the **learning and skill-building phase**, preparing to enter the 

### Prompt with some key

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Responsiblity : Make sure not to give think content in your response , Act like a professional Q&A Analyst and answer all the question as asked by user always answer in {language}"),
        MessagesPlaceholder("message")
    ]
)
chain = prompt | model

In [ ]:
chain.invoke(
    {"message" : [HumanMessage("I am Amber Khan an Indian Citizen interest in sports not in polictics")],
     "language" : "Hindi"}
    )

AIMessage(content='<think>\n\n</think>\n\nहार्दिक बधाई, अम्बर खान जी! 😊  \nआपका रुचि खेल में होना बहुत प्रशंसनीय है। खेल एक ऐसा क्षेत्र है जो ऊर्जा, आत्मविश्वास और टीम वर्क को बढ़ावा देता है। आप अपने खेल के क्षेत्र में अपना जज्बा और दृढ़ता से बहुत कुछ ले सकते हैं।  \n\nक्या आप किसी विशेष खेल में रुचि रखते हैं जैसे क्रिकेट, सॉकर, बैडमिंटन, बास्केटबॉल, आदि? हम आपकी रुचि के आधार पर कुछ टिप्स या रणनीति भी शेयर कर सकते हैं।  \n\nआपको खेल में अपना सर्वश्रेष्ठ प्रदर्शन करने के लिए कोई विशेष प्रशिक्षण, सुविधाएँ या अनुमानित निर्देश चाहिए? हम आपके लिए संसाधन भी ढूंढ़ सकते हैं।  \n\nकृपया हमें बताएं कि आप अपने खेल जीवन में कैसे आगे बढ़ना चाहते हैं। हम आपकी पूरी तरह से सहायता करेंगे। 🏀⚽🏏', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 596, 'prompt_tokens': 64, 'total_tokens': 660, 'completion_time': 1.56234895, 'completion_tokens_details': None, 'prompt_time': 0.002350419, 'prompt_tokens_details': None, 'queue_time': 0.15929064, 'total_time': 1.564699369}, 'model_na

In [ ]:
chat_with_history = RunnableWithMessageHistory(chain, session_wise_history, input_messages_key="message")

configuration5 = {"configurable" : {"session_id" : "chat5"}}

chat_with_history.invoke( 
    {"message" : [HumanMessage("I am Amber Khan an Indian Citizen interest in sports not in polictics")], "language" : "Hindi"},
    configuration5
)

AIMessage(content='<think>\n\n</think>\n\nअम्बर खान, आपका स्पोर्ट्स में रुचि होना बहुत अच्छी बात है। क्या आपने कोई विशेष खेल खेला है या फिर आप किस खेल में रुचि रखते हैं? अगर आपको खेलों से जुड़ी कोई अपडेट, प्रतियोगिताएं, ट्रेनिंग या करियर बनाने के तरीके जानने हों तो मुझे बताएं, मैं आपकी मदद करने की कोशिश करूंगा।', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 256, 'prompt_tokens': 64, 'total_tokens': 320, 'completion_time': 0.529743109, 'completion_tokens_details': None, 'prompt_time': 0.002824085, 'prompt_tokens_details': None, 'queue_time': 0.320203512, 'total_time': 0.532567194}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_efa9879028', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--efafe917-cfba-40e9-8ac2-24fede6cecf5-0', usage_metadata={'input_tokens': 64, 'output_tokens': 256, 'total_tokens': 320})

In [ ]:
chat_with_history.invoke(
    {"message" : [HumanMessage("what is my name")], "language" : "English"},
    configuration5
)

AIMessage(content='<think>\n\n</think>\n\nYour name is **Amber Khan**, as you introduced yourself at the beginning of the conversation.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 592, 'total_tokens': 616, 'completion_time': 0.036709446, 'completion_tokens_details': None, 'prompt_time': 0.027101181, 'prompt_tokens_details': None, 'queue_time': 0.160547099, 'total_time': 0.063810627}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--83d94b1e-23f0-4f81-96fc-17f42cf02358-0', usage_metadata={'input_tokens': 592, 'output_tokens': 24, 'total_tokens': 616})